In [1]:
import h5py 
import numpy as np 
from autolab_core import RigidTransform
from scipy.spatial.transform import Rotation as R

In [2]:
file = h5py.File("/home/mfu/dataset/dp_gs/pick_tiger_241123/robot_trajectory_2024_11_23_19_24_18.h5")

In [3]:
# action = file["action/cartesian_pose"][:, :7]
# proprio = file["state/cartesian/cartesian_pose"][:, :7]

action = file["state/cartesian/cartesian_pose"][16:, 7:]
proprio = file["state/cartesian/cartesian_pose"][:-16, 7:]

In [4]:
def wxyz_xyz2rt(wxyz_xyz):
    wxyz = wxyz_xyz[:4]
    xyz = wxyz_xyz[4:]
    rot = R.from_quat(wxyz, scalar_first=True)
    return RigidTransform(rot.as_matrix(), xyz, from_frame='gripper', to_frame='world')

In [5]:
action = [wxyz_xyz2rt(a) for a in action]
proprio = [wxyz_xyz2rt(p) for p in proprio]

In [6]:
# calculate delta in world frame 
delta = []
for i in range(len(action)):
    delta.append(action[i] * (proprio[i].inverse()))
print(delta[0].from_frame, delta[0].to_frame)
delta_trans = np.stack([d.translation for d in delta])
min_delta, max_delta = delta_trans.min(0), delta_trans.max(0)
print(min_delta, max_delta)

world world
[-4.54347369e-02 -8.83773183e-01 -9.52171430e-05] [0.06379791 0.49193136 0.87625758]


In [7]:
# calculate delta in gripper frame 
delta = []
for i in range(len(action)):
    delta.append(proprio[i].inverse() * action[i])
print(delta[0].from_frame, delta[0].to_frame)
delta_trans = np.stack([d.translation for d in delta])
min_delta, max_delta = delta_trans.min(0), delta_trans.max(0)
print(min_delta, max_delta)

gripper gripper
[-0.08520899 -0.27982753 -0.38598569] [1.15306834e-01 3.75039294e-01 6.20057240e-05]
